<a href="https://colab.research.google.com/github/athiq-ahmed/Pytorch/blob/master/Captcha_recognition_using_PyTorch_(Convolutional_RNN_%2B_CTC_Loss).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl -LO https://github.com/AakashKumarNain/CaptchaCracker/raw/master/captcha_images_v2.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   159  100   159    0     0    210      0 --:--:-- --:--:-- --:--:--   210
100 8863k  100 8863k    0     0  5584k      0  0:00:01  0:00:01 --:--:-- 48.3M


In [2]:
!unzip -qq captcha_images_v2.zip